In [5]:
import igraph
import networkx as nx
import numpy as np
import pandas as pd

In [13]:
dim = [2, 4]
g = igraph.Graph.Lattice(dim, nei=1, directed=False, mutual=True, circular=True)
for v in g.vs: 
    r = np.random.rand() 
    v['nparticles'] = round(r*10)
    mycolor = list([0,0,r])
    v['color'] = mycolor
    
visual = {}
# visual["layout"] = g.layout('grid')
visual["bbox"] = (200, 200)
visual["margin"] = 10
igraph.plot(g, **visual)

<function igraph.write(graph, filename, *args, **kwds)>

In [103]:
dim = [4, 4]
g = igraph.Graph.Lattice(dim, nei=1, directed=False, mutual=True, circular=True)


for v in g.vs:  # initialize all nodes with 10 particles
    v['nparticles'] = 100

    
nepochs = 10

############ set attractiviness
g.vs['attractiviness'] = 5
g.vs[1]['attractiviness'] = 1
g.vs[3]['attractiviness'] = 25

########### simulation
for ep in range(nepochs):
    allparticles = [ v['nparticles'] for v in g.vs ] # copy to avoid being altered
    for i, v in enumerate(g.vs):
        n = allparticles[i]
        neighids = g.neighbors(i)
        attractiviness = g.vs[neighids]['attractiviness']
        attractiviness /= np.sum(attractiviness)

        for p in range(n):
            neighid = np.random.choice(neighids, p=attractiviness)
            v['nparticles'] -= 1
            g.vs[neighid]['nparticles'] += 1
#             print(i, neighid, g.vs['nparticles'])

#     for i, v in enumerate(g.vs): # dynamics loop
        

############ plot
# for v in g.vs: 
#     v['color'] = [0, 0, v['nparticles']/10]
    
visual = {}
# visual["layout"] = g.layout('grid')
visual["bbox"] = (200, 200)
visual["margin"] = 10
# labels = [str(v['nparticles']) for v in g.vs]
igraph.plot(g, vertex_label=g.vs['nparticles'], **visual)

IndentationError: expected an indented block (<ipython-input-103-6294b112424d>, line 38)